# Facial Expression Recognition Workflow (Ubuntu Development Environment)

In [1]:
# !pip install mediapipe opencv-python matplotlib gradio tensorflow keras

## Mediapipe Face Detection + Bounding Box Crop

In [2]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_face(image_path):
    img = cv2.imread(image_path)
    mp_face = mp.solutions.face_detection
    face_detection = mp_face.FaceDetection(model_selection=0, min_detection_confidence=0.5)
    results = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    bbox = None
    if results.detections:
        for det in results.detections:
            bboxC = det.location_data.relative_bounding_box
            h, w, _ = img.shape
            bbox = int(bboxC.xmin * w), int(bboxC.ymin * h), \
                   int(bboxC.width * w), int(bboxC.height * h)
            face_crop = img[bbox[1]:bbox[1]+bbox[3], bbox[0]:bbox[0]+bbox[2]]
            return img, face_crop, bbox
    return img, None, None


2025-04-16 23:34:38.163018: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-16 23:34:41.366917: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744860882.402566   80411 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744860882.655409   80411 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744860884.939560   80411 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

## Load FER-2013 Emotion Classification Model (Keras Example)

In [ ]:
from tensorflow.keras.models import load_model

# Load model and make predictions
def predict_emotion(face_crop, model_path='./model/fer_model_best.h5'):
    model = load_model(model_path)
    input_face = cv2.cvtColor(face_crop, cv2.COLOR_BGR2GRAY)
    input_face = cv2.resize(input_face, (48, 48)) / 255.0
    input_face = input_face.reshape(1, 48, 48, 1)
    prediction = model.predict(input_face)
    return prediction


## Visualize（OpenCV / matplotlib）

In [4]:
def visualize_prediction(img, bbox, prediction):
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    pred_label = emotion_labels[np.argmax(prediction)]
    
    if bbox:
        cv2.rectangle(img, (bbox[0], bbox[1]),
                      (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 255, 0), 2)
        cv2.putText(img, pred_label, (bbox[0], bbox[1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

## Bonus: Gradio Graphical Interface

In [5]:
import gradio as gr

def gradio_interface(image):
    image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imwrite("temp.jpg", image_bgr)
    img, face_crop, bbox = detect_face("temp.jpg")
    if face_crop is not None:
        pred = predict_emotion(face_crop)
        visualize_prediction(img, bbox, pred)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

gr.Interface(fn=gradio_interface,
             inputs=gr.Image(type="numpy"),
             outputs=gr.Image(type="numpy"),
             title="Facial Expression Classifier").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
